# 1. Dependencies

In [42]:
!pip install tensorflow tensorflow-gpu opencv-python mediapipe sklearn matplotlib

# Steps to run
1. test whether opencv2 works with your camera
2. then collect your own dataset of the actions
3. run model and save
4. try testing in real time

In [43]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [44]:
mp_holistic = mp.solutions.holistic  # Holistic model
mp_drawing = mp.solutions.drawing_utils  # Drawing utilities


def mediapipe_detection(image, model):
    # COLOR CONVERSION BGR 2 RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # COLOR COVERSION RGB 2 BGR
    return image, results


def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)      # Draw face
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)          # Draw pose
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)     # Draw left hand
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS)    # Draw right hand


def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                              mp_drawing.DrawingSpec(
                                  color=(80, 110, 10), thickness=1, circle_radius=1),
                              mp_drawing.DrawingSpec(
                                  color=(80, 256, 121), thickness=1, circle_radius=1)
                              )
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(
                                  color=(80, 22, 10), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(
                                  color=(80, 44, 121), thickness=2, circle_radius=2)
                              )
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(
                                  color=(121, 22, 76), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(
                                  color=(121, 44, 250), thickness=2, circle_radius=2)
                              )
    # Draw right hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(
                                  color=(245, 117, 66), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(
                                  color=(245, 66, 230), thickness=2, circle_radius=2)
                              )


In [45]:
# # Test capture, press q to quit
# cap = cv2.VideoCapture(0)

# with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
#     while cap.isOpened():

#         # Read feed
#         ret, frame = cap.read()

#         # Make detections
#         image, results = mediapipe_detection(frame, holistic)
        
#         # Draw landmarks
#         draw_styled_landmarks(image, results)

#         # Show to screen
#         cv2.imshow('OpenCV Feed', image)

#         # Break gracefully
#         if cv2.waitKey(10) & 0xFF == ord('q'):
#             break
#     cap.release()
#     cv2.destroyAllWindows()

In [46]:
# # draw landmarks on frame and show on matplotlib
# draw_landmarks(frame, results)
# # print length of landmarks if left hand in frame else return error
# len(results.left_hand_landmarks.landmark) if results.left_hand_landmarks else print("left hand not detected")

# plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))


# Extract Keypoint Values

In [134]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten(
    ) if results.pose_landmarks else np.zeros(132)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten(
    ) if results.face_landmarks else np.zeros(1404)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten(
    ) if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten(
    ) if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [133]:
# # extract keypoints and save in numpy file
result_test = extract_keypoints(results)
# np.save(os.path.join(DATA_PATH, "hello", '1', '1'), result_test)

468
(1404,)


# Collect Keypoint Values for Training and Testing

In [162]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('.', 'data') 

# Actions that we try to detect
actions = np.array(['hello', 'thanks', 'iloveyou', 'nothing', 'cannot', 'dinner'])

# Videos are going to be 30 frames in length
sequence_length = 30

In [163]:
# create folder
def create_folders(action, sequence_no):
    if action not in actions:
        np.append(actions, action)
    for sequence in range(1, sequence_no + 1):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# trains specific action, set starting_folder to prevent overwrite
# press q to go next
# hold q to end
def train_action(action, starting_folder=1, sequence_no=30):
    create_folders(action, sequence_no)

    cap = cv2.VideoCapture(0)
    # Set mediapipe model 
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

        # Loop through videos
        for sequence in range(starting_folder, starting_folder + sequence_no):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (100, 200), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(500)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                            cv2.FONT_HERSHEY_PLAIN, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Skip to next recording
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
            # Break out of window entirely
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
                    
        cap.release()
        cv2.destroyAllWindows()

In [138]:
train_action("cannot")

In [90]:
# # end recording forcefully
# cap.release()
# cv2.destroyAllWindows()

# Create Labels and Features

In [109]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [164]:
label_map = {label:num for num, label in enumerate(actions)}

sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [168]:
np.array(sequences).shape

(180, 30, 1662)

In [166]:

X = np.array(sequences)
y = to_categorical(labels).astype(int)

In [169]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

# Build and Train LSTM Neural Network

In [170]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [171]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [172]:
# delete previous model
del model

In [173]:
# Build model
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax')) # turn into array of probabilities of action that sum to 1
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [174]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
6/6 [==============================] - 3s 143ms/step - loss: 1.8585 - categorical_accuracy: 0.1605
Epoch 2/2000
6/6 [==============================] - 0s 58ms/step - loss: 3.1379 - categorical_accuracy: 0.2099
Epoch 3/2000
6/6 [==============================] - 0s 54ms/step - loss: 1.8020 - categorical_accuracy: 0.1173
Epoch 4/2000
6/6 [==============================] - 0s 56ms/step - loss: 1.8079 - categorical_accuracy: 0.1728
Epoch 5/2000
6/6 [==============================] - 0s 58ms/step - loss: 1.8202 - categorical_accuracy: 0.1667
Epoch 6/2000
6/6 [==============================] - 0s 54ms/step - loss: 1.8086 - categorical_accuracy: 0.1420
Epoch 7/2000
6/6 [==============================] - 0s 55ms/step - loss: 1.8122 - categorical_accuracy: 0.1790
Epoch 8/2000
6/6 [==============================] - 0s 53ms/step - loss: 1.7701 - categorical_accuracy: 0.1728
Epoch 9/2000
6/6 [==============================] - 0s 54ms/step - loss: 1.7445 - categorical_accuracy: 0.2469


KeyboardInterrupt: 

In [151]:
# save model
model.save('action2.h5')

In [63]:
# continue training
model.load_weights('action2.h5')

In [175]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_18 (LSTM)               (None, 30, 64)            442112    
_________________________________________________________________
lstm_19 (LSTM)               (None, 30, 128)           98816     
_________________________________________________________________
lstm_20 (LSTM)               (None, 64)                49408     
_________________________________________________________________
dense_18 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_19 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_20 (Dense)             (None, 6)                 198       
Total params: 596,774
Trainable params: 596,774
Non-trainable params: 0
________________________________________________

# Make Predictions

In [176]:
res = model.predict(X_test)

In [177]:
actions[np.argmax(res[4])]

'thanks'

In [178]:
actions[np.argmax(y_test[4])]

'iloveyou'

In [179]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [180]:
yhat = model.predict(X_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

multilabel_confusion_matrix(ytrue, yhat)

array([[[14,  0],
        [ 4,  0]],

       [[ 0, 15],
        [ 0,  3]],

       [[16,  0],
        [ 2,  0]],

       [[16,  0],
        [ 2,  0]],

       [[15,  0],
        [ 3,  0]],

       [[14,  0],
        [ 4,  0]]], dtype=int64)

In [181]:
accuracy_score(ytrue, yhat)

0.16666666666666666

# Test in Real Time

In [158]:
from scipy import stats

In [159]:
def random_color():
    tmp = np.random.randint(0, 255, size=3)
    return (int(tmp[0]), int(tmp[1]), int(tmp[2]))

In [160]:
colors = [random_color() for x in range(len(actions))]

def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [182]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            # print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

IndexError: list index out of range

In [183]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, cv2.cvtColor(image, cv2.COLOR_BGR2RGB), colors))

IndexError: list index out of range

<Figure size 1296x1296 with 0 Axes>

# Export model

In [76]:
print(f"tensorflowjs_converter --input_format keras action.h5 exported")

tensorflowjs_converter --input_format keras action.h5 exported
